In [6]:
from bs4 import BeautifulSoup

path = 'produced_data/header.xml'

with open(path, 'r', encoding='utf-8') as f:
    def retrieve_person(xml_data):
        soup = BeautifulSoup(xml_data, 'xml')

        # Get all the person elements.
        persons = soup.find_all('person')

        # Extract the XML IDs and value labels.
        xml_ids_and_value_labels = [(person['xml:id'], person.find('persName').text) for person in persons]

        return xml_ids_and_value_labels
    
    def retrieve_places(xml_data):
        soup = BeautifulSoup(xml_data, 'xml')

        # Get all the person elements.
        places = soup.find_all('place')

        # Extract the XML IDs and value labels.
        xml_ids_and_value_labels = [{place.find('placeName').text: place['xml:id']} for place in places]

        return xml_ids_and_value_labels

# Example usage:

    xml_ids_and_value_labels = retrieve_places(f)

    print(xml_ids_and_value_labels)

[{'Urbino': 'Urbino'}, {'Europa': 'Europe'}, {'Genova': 'Genoa'}, {'Nemours': 'Nemours'}, {'Grecia': 'Greece'}, {'Venezia': 'Venice'}, {'Napoli': 'Naples'}, {'Bologna': 'Bologna'}, {'Museo Civico Medievale di Bologna': 'MCMBologna'}, {'Uffizi': 'Uffizi'}, {'basilica di San Petronio': 'BSanPetronio'}, {'Egitto': 'Egypt'}, {'Palazzo Bolognini Amorini': 'palazzo-salina-amorini'}, {'Italia': 'Italy'}, {'Cremona': 'Cremona'}, {'National Gallery': 'NGLondon'}, {'Roma': 'Rome'}, {'Musei Vaticani': 'VaticanMuseums'}, {'Madrid': 'Madrid'}, {'Museo del Prado': 'PradoMuseum'}, {'Palermo': 'Palermo'}, {'Sicilia': 'Sicily'}, {'Isola di Capri': 'Capri'}, {'monastero della Santissima Annunziata\n': 'Annunziata-Paternò'}, {'Imola': 'Imola'}, {'Parigi': 'Paris'}, {'Francia': 'France'}, {'Pisa': 'Pisa'}, {'Firenze': 'Florence'}, {'Hampton Court': 'HCourt'}, {'chiesa di San Lorenzo in Fonte': 'sanlinfonte'}, {'Accademia nazionale di San Luca': 'accademia-san-luca-roma'}, {'basilica di San Domenico': 'san

In [ ]:
from realNameList import personList
from placeList import placeList
from dictworklocation import worklocationdict
import sparql_dataframe
import pandas

#store the people missing from wikidata and the multiple entries
missingpeople=[]
multiplepeople=[]

placeListLabel = list(key for dictionary in placeList for key in dictionary)       

def personRDF(slicer1, slicer2):
       result = []        
       for x,y in personList[slicer1:slicer2]:
              #result=""
              name = y

              endpoint = 'https://query.wikidata.org/sparql'
              
              queryPerson = """
              SELECT ?human ?humanLabel ?genderLabel ?birthDate ?birthPlace ?birthPlaceLabel ?deathDate ?deathPlace ?deathPlaceLabel ?occupation ?occupationLabel ?workLocation ?workLocationLabel

              WHERE {
              ?human wdt:P31 wd:Q5;  # Instances of human (Q5)
                     rdfs:label "%s"@it.  # Label in Italian
              ?human rdfs:label ?humanLabel.  # Get the label
              FILTER(LANG(?humanLabel) = "it")  # Filter for Italian labels

              OPTIONAL {?human wdt:P21 ?gender.
                     ?gender rdfs:label ?genderLabel.
                     FILTER(LANG(?genderLabel) = "it")}
              OPTIONAL{?human wdt:P569 ?birthDate.}
              OPTIONAL{?human wdt:P19 ?birthPlace.
                     ?birthPlace rdfs:label ?birthPlaceLabel.
                     FILTER(LANG(?birthPlaceLabel) = "it")}
              OPTIONAL{?human wdt:P570 ?deathDate.}
              OPTIONAL{?human wdt:P20 ?deathPlace.
                     ?deathPlace rdfs:label ?deathPlaceLabel.
                     FILTER(LANG(?deathPlaceLabel) = "it") }
              OPTIONAL{?human wdt:P106 ?occupation.
                     ?occupation rdfs:label ?occupationLabel.
                     FILTER(LANG(?occupationLabel) = "it")}
              OPTIONAL {?human wdt:P937 ?workLocation.
                     ?workLocation rdfs:label ?workLocationLabel.
                     FILTER(LANG(?workLocationLabel) = "it")}

              }
              """%(name)

              df = sparql_dataframe.get(endpoint, queryPerson, post='True')



              #check if there's only one person in wikidata
              
              same = df['human'].unique()
              
              if df.empty:
                     endpoint = 'https://query.wikidata.org/sparql'
              
                     queryPerson = """
              SELECT ?human ?humanLabel ?genderLabel ?birthDate ?birthPlace ?birthPlaceLabel ?deathDate ?deathPlace ?deathPlaceLabel ?occupation ?occupationLabel ?workLocation ?workLocationLabel

              WHERE {
              ?human wdt:P31 wd:Q5;  # Instances of human (Q5)
                     rdfs:label "{id}"@en.  # Label in Italian
              ?human rdfs:label ?humanLabel.  # Get the label
              FILTER(LANG(?humanLabel) = "en")  # Filter for Italian labels

              OPTIONAL {?human wdt:P21 ?gender.
                     ?gender rdfs:label ?genderLabel.
                     FILTER(LANG(?genderLabel) = "it")}
              OPTIONAL{?human wdt:P569 ?birthDate.}
              OPTIONAL{?human wdt:P19 ?birthPlace.
                     ?birthPlace rdfs:label ?birthPlaceLabel.
                     FILTER(LANG(?birthPlaceLabel) = "it")}
              OPTIONAL{?human wdt:P570 ?deathDate.}
              OPTIONAL{?human wdt:P20 ?deathPlace.
                     ?deathPlace rdfs:label ?deathPlaceLabel.
                     FILTER(LANG(?deathPlaceLabel) = "it") }
              OPTIONAL{?human wdt:P106 ?occupation.
                     ?occupation rdfs:label ?occupationLabel.
                     FILTER(LANG(?occupationLabel) = "it")}
              OPTIONAL {?human wdt:P937 ?workLocation.
                     ?workLocation rdfs:label ?workLocationLabel.
                     FILTER(LANG(?workLocationLabel) = "it")}

              }
              """%(name)

                     df = sparql_dataframe.get(endpoint, queryPerson, post='True')

                     if df.empty:

                            missingpeople.append((x,y))
              
              elif len(same)>1:
                     multiple_entries= same.tolist()
                     multiplepeople.append(multiple_entries)


              else:
                     birthPlaceLabel = df['birthPlaceLabel'][0] 
                     deathPlaceLabel = df['deathPlaceLabel'][0]
                     createrdf = """
<rdf:Description tei:ref='{id}' rdf:about='{wikidataentity}'>
       <rdf:type rdf:resource="http://xmlns.com/foaf/0.1/Person"/>
       <rdf:label xml:lan="it">{label}</rdf:label>
       <crm:P61_has_gender>{genderLabel}</crm:P61_has_gender>
       <schema:birthDate rdf:datatype="http://www.w3.org/2001/XMLSchema#date">{birthDate}</schema:birthDate>
       <schema:deathDate rdf:datatype="http://www.w3.org/2001/XMLSchema#date">{deathDate}</schema:deathDate>\n""".format(id=x, wikidataentity=df['human'][0], label=df['humanLabel'][0], genderLabel=df['genderLabel'][0], birthDate=df['birthDate'][0], deathDate=df['deathDate'][0])
                     
              #add birth place as referenced in the header or as entity in wikidata
              
                     if birthPlaceLabel in placeList:
                     
                            birthPlaceLabelId= placeList[birthPlaceLabel]

              # #ho dei dubbi sul primo caso

                            createrdf = createrdf + '\t<schema:birthPlace xml:ref="{id}">{id}</schema:birthPlace>\n'%(birthPlaceLabelId, birthPlaceLabel)

                     else:
                            createrdf = createrdf + '\t<schema:birthPlace rdf:resource="{id}">{id}<schema:birthPlace>\n'%(df['birthPlace'][0], df['birthPlaceLabel'][0])

              # #add death place as referenced in the header or as entity in wikidata

                     if deathPlaceLabel in placeList:

                            deathPlaceLabelId= placeList[deathPlaceLabel]

              # #ho dei dubbi sul primo caso
                            createrdf = createrdf + '\t<schema:deathPlace rdf:resource="{id}">{id}</schema:deathPlace>\n'%(deathPlaceLabelId, deathPlaceLabel)

                     else:
                            createrdf = createrdf + '\t<schema:deathPlace rdf:resource="%s">%s</schema:deathPlace>\n'%(df['deathPlace'][0], df['deathPlaceLabel'][0])

                     if df['occupation'][0] == 'Nan':
                            for occupation in df['occupation'].unique():
                                   occupationLabel = df.loc[df['occupation'] == occupation, 'occupationLabel'].values[0]
                                   createrdf = createrdf + '<schema:hasOccupation rdf:resource="%s">%s</schema:hasOccupation>\n'%(occupation, occupationLabel)
                     
                     if df['humanLabel'][0] in worklocationdict.keys():
                            for i in worklocationdict[df['humanLabel'][0]]:
                                   if i in placeListLabel:
                                          if i in df['workLocationLabel'].unique():

                                                 worklocationlabel = df.loc[df['workLocationLabel'] == i, 'workLocation'].values[0]
                                                 createrdf = createrdf + '\t<schema:workLocation rdf:resource"%s">%s</schema:workLocation>\n'%(worklocationlabel, i)
                     
                     
                     
                     
                     createrdf = createrdf + "</rdf:Description>"
                     result.append(createrdf)

       print(missingpeople, multiplepeople)
       print(len(missingpeople), len(multiplepeople))
       print(len(result))
       return '\n'.join(result)

result = []
slicer1 = 0
slicer2 = 540//20
for i in range(540//20):
       rdf = personRDF(slicer1,slicer2)
       result.append(rdf)
       slicer1 = slicer1 + (540//20)
       slicer2 = slicer2 + (540//20)
       print(rdf)

print(result)

In [10]:
#query for places
import sparql_dataframe
from placeList import placeList

def placeRDF(slice1, slice2):
       result = []
       missingplace=[]
       multipleplace=[]
       for i in placeList[slice1:slice2]:

              label = list(i.items())
              placelabel = label[0][0]
              placelabelid = label[0][1]
              

              endpoint = 'https://query.wikidata.org/sparql'

              queryPlace= """SELECT ?place ?placeLabel ?coordinate  WHERE {
              ?place wdt:P31/wdt:P279* wd:Q2221906; # Instances of place (Q2221906) or its subclasses
                     rdfs:label "%s"@it; # Label in English
                     wdt:P625 ?coordinate.
              ?place rdfs:label ?placeLabel. # Get the label
              FILTER(LANG(?placeLabel) = "en") # Filter for English labels
              }
              """%placelabel
              
              df = sparql_dataframe.get(endpoint, queryPlace, post='True')
              
              same = df['place'].unique()

              if df.empty:
                     missingplace.append(i)
              
              elif len(same)>1:
                     multiple_entries= same.tolist()
                     multiplepeople.append(multiple_entries)
              
              else:

                     coordinate = df['coordinate'][0]
                     cleancoordinate = coordinate[6:-1]
                     finalcoordinate = cleancoordinate.split(' ')

                     createrdf ="""
              <rdf:Description tei:ref="{id}" rdf:about="{about}">
                     <rdf:type rdf:resource="{wid}"/>
                     <rdfs:label xml:lang="en">{label}</rdfs:label>
                     <geo:lat>{lat}</geo:lat>
                     <geo:long>{long}</geo:long>
              </rdf:Description>

              """.format(id=placelabelid, wid = df['place'][0], label = placelabel, lat = finalcoordinate[1], long = finalcoordinate[0], about = df['place'][0])
       
                     result.append(createrdf)
       
       
       print(missingplace, multipleplace)
       return result 

print(placeRDF(0,10))

[{'Museo Civico Medievale di Bologna': 'MCMBologna'}] []
['\n              <rdf:Description tei:ref="Urbino" rdf:about="http://www.wikidata.org/entity/Q2759">\n                     <rdf:type rdf:resource="http://www.wikidata.org/entity/Q2759"/>\n                     <rdfs:label xml:lang="en">Urbino</rdfs:label>\n                     <geo:lat>43.725238888</geo:lat>\n                     <geo:long>12.6372</geo:long>\n              </rdf:Description>\n\n              ', '\n              <rdf:Description tei:ref="Nemours" rdf:about="http://www.wikidata.org/entity/Q321067">\n                     <rdf:type rdf:resource="http://www.wikidata.org/entity/Q321067"/>\n                     <rdfs:label xml:lang="en">Nemours</rdfs:label>\n                     <geo:lat>48.268611111</geo:lat>\n                     <geo:long>2.693611111</geo:long>\n              </rdf:Description>\n\n              ', '\n              <rdf:Description tei:ref="Uffizi" rdf:about="http://www.wikidata.org/entity/Q51252">\n 

In [1]:
from workList import workList
import sparql_dataframe

# def workRDF(slicer1,slicer2):
result = []
for x in list(workList.keys())[0:1]:
    
    id = 'wd:Q175036'
    
    endpoint = 'https://query.wikidata.org/sparql'
    
    query = """select distinct ?author ?authorLabel ?title ?inception ?creationLocation ?creationLocationLabel ?creationLocationCoordinate ?depicts ?depictsLabel ?movement ?movementLabel ?genre ?genreLabel ?location ?locationLabel ?locationCoordinate where{
    optional{%s wdt:P50 | wdt:P170 ?author.
            ?author rdfs:label ?authorLabel.
            FILTER(LANG(?authorLabel) = "it")}
    optional{%s wdt:P1476 ?title. }
    optional{%s wdt:P135 ?movement.
            ?movement rdfs:label ?movementLabel. 
            FILTER(LANG(?movementLabel) = "it")}
    optional{%s wdt:P136 ?genre.
            ?genre rdfs:label ?genreLabel.
            FILTER(LANG(?genreLabel) = "it")}           
    optional{%s wdt:P571 ?inception .}      
    optional{%s wdt:P180 ?depicts .
            ?depicts rdfs:label ?depictsLabel.
            FILTER(LANG(?depictsLabel) = "it")} 
    optional{%s wdt:P276 ?location.
            ?location rdfs:label ?locationLabel.
            FILTER(LANG(?locationLabel) = "it")}
optional{?location wdt:P625 ?locationCoordinate.}
optional{%s wdt:P1071 ?creationLocation.
            ?creationLocation rdfs:label ?creationLocationLabel.
            FILTER(LANG(?creationLocationLabel) = "it")}
optional{?creationLocation wdt:P625 ?creationLocationCoordinate. }
    }"""%(id,id,id,id,id,id,id,id)

    df = sparql_dataframe.get(endpoint, query, post='True')

    createrdf = """<rdf:Description tei:ref="{id}" rdf:about="{about}">"""


df

,author,authorLabel,title,inception,creationLocation,creationLocationLabel,creationLocationCoordinate,depicts,depictsLabel,movement,movementLabel,genre,genreLabel,location,locationLabel,locationCoordinate
0,http://www.wikidata.org/entity/Q5593,Pablo Picasso,Guernica,1937-01-01T00:00:00Z,http://www.wikidata.org/entity/Q90,Parigi,Point(2.352222222 48.856666666),http://www.wikidata.org/entity/Q170351,bombardamento di Guernica,http://www.wikidata.org/entity/Q42934,cubismo,http://www.wikidata.org/entity/Q742333,pittura storica,http://www.wikidata.org/entity/Q460889,Museo Nacional Centro de Arte Reina Sofía,Point(-3.693996097 40.408571429)
1,http://www.wikidata.org/entity/Q5593,Pablo Picasso,Guernica,1937-01-01T00:00:00Z,http://www.wikidata.org/entity/Q90,Parigi,Point(2.352222222 48.856666666),http://www.wikidata.org/entity/Q10859,guerra civile spagnola,http://www.wikidata.org/entity/Q42934,cubismo,http://www.wikidata.org/entity/Q742333,pittura storica,http://www.wikidata.org/entity/Q460889,Museo Nacional Centro de Arte Reina Sofía,Point(-3.693996097 40.408571429)
2,http://www.wikidata.org/entity/Q5593,Pablo Picasso,Guernica,1937-01-01T00:00:00Z,http://www.wikidata.org/entity/Q90,Parigi,Point(2.352222222 48.856666666),http://www.wikidata.org/entity/Q170351,bombardamento di Guernica,http://www.wikidata.org/entity/Q39427,surrealismo,http://www.wikidata.org/entity/Q742333,pittura storica,http://www.wikidata.org/entity/Q460889,Museo Nacional Centro de Arte Reina Sofía,Point(-3.693996097 40.408571429)
3,http://www.wikidata.org/entity/Q5593,Pablo Picasso,Guernica,1937-01-01T00:00:00Z,http://www.wikidata.org/entity/Q90,Parigi,Point(2.352222222 48.856666666),http://www.wikidata.org/entity/Q10859,guerra civile spagnola,http://www.wikidata.org/entity/Q39427,surrealismo,http://www.wikidata.org/entity/Q742333,pittura storica,http://www.wikidata.org/entity/Q460889,Museo Nacional Centro de Arte Reina Sofía,Point(-3.693996097 40.408571429)
